# CVDDivergence Strategy Analysis

This notebook provides an in-depth analysis of the CVDDivergence strategy, including its logic, indicators, and backtest results.

## Setup

### Change Working directory to repository root

In [1]:
from constants import set_cwd_to_project_root, CONFIG_DIR
from main import generate_config

generate_config("binance")

set_cwd_to_project_root()

### Configure Freqtrade environment

In [2]:
from freqtrade.configuration import Configuration


config = Configuration.from_files([CONFIG_DIR])

config.update(
    {
        "timeframe": "5m",
        "strategy": "CVDDivergence"
    }
)

# Pair to analyze - Only use one pair here
PAIR = "BTC/USDT:USDT"

In [3]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from constants import DATA_DIR

candles = load_pair_history(
    datadir=DATA_DIR / "binance",
    timeframe=config["timeframe"],
    pair=PAIR,
    data_format="feather",
    candle_type=CandleType.FUTURES,
)

# Confirm success
print(f"Loaded {len(candles)} rows of data for {PAIR} from {DATA_DIR}")
candles.head()

Loaded 77949 rows of data for BTC/USDT:USDT from F:\quantative-analysis\data


,date,open,high,low,close,volume
0,2024-01-01 00:00:00+00:00,42314.0,42437.2,42289.6,42437.1,1724.210
1,2024-01-01 00:05:00+00:00,42437.2,42474.1,42420.5,42446.8,994.003
2,2024-01-01 00:10:00+00:00,42446.8,42535.0,42445.2,42532.5,899.775
3,2024-01-01 00:15:00+00:00,42532.4,42603.2,42494.1,42494.1,1291.232
4,2024-01-01 00:20:00+00:00,42494.1,42533.1,42484.4,42509.4,418.463


## Load and run strategy

In [4]:
# Load strategy using values set above
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": PAIR})
df.tail()

,date,open,high,low,close,volume,trades,orderflow,imbalances,stacked_imbalances_bid,...,bear_poc_upper,atr,enter_tag,enter_long,enter_short,atr_stoploss_long,atr_stoploss_short,exit_tag,exit_long,exit_short
77944,2024-09-27 15:20:00+00:00,66342.0,66388.8,66270.0,66302.3,1052.536,NaN,NaN,NaN,NaN,...,65121.12,159.052035,,NaN,NaN,65984.195931,66620.404069,VWAP_Upper,1.0,1.0
77945,2024-09-27 15:25:00+00:00,66302.3,66330.0,66222.3,66303.1,657.397,NaN,NaN,NaN,NaN,...,65121.12,155.384032,,NaN,NaN,65992.331936,66613.868064,,NaN,NaN
77946,2024-09-27 15:30:00+00:00,66303.2,66322.0,66231.0,66258.5,673.062,NaN,NaN,NaN,NaN,...,65121.12,150.785173,,NaN,NaN,65956.929655,66560.070345,,NaN,NaN
77947,2024-09-27 15:35:00+00:00,66258.6,66276.0,66183.4,66200.0,610.517,NaN,NaN,NaN,NaN,...,65121.12,146.629089,,NaN,NaN,65906.741822,66493.258178,,NaN,NaN
77948,2024-09-27 15:40:00+00:00,66200.0,66289.0,66158.4,66170.2,749.706,NaN,NaN,NaN,NaN,...,65121.12,145.484154,,NaN,NaN,65879.231692,66461.168308,,NaN,NaN


### Display the trade details

In [5]:
# Report results
print(f"Generated {df['enter_long'].sum()} long entry signals")
print(f"Generated {df['enter_short'].sum()} short entry signals")
data = df.set_index("date", drop=False)

Generated 8.0 long entry signals
Generated 12.0 short entry signals


## Load backtest results

In [6]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
from constants import BACKTEST_RESULTS_DIR

# Load backtest statistics
stats = load_backtest_stats(BACKTEST_RESULTS_DIR)

strategy = "CVDDivergence"
# Example usages:
print("Results per pair:")
print(stats["strategy"][strategy]["results_per_pair"])
print("\nPairlist:")
print(stats["strategy"][strategy]["pairlist"])
print(f"\nMarket change: {stats['strategy'][strategy]['market_change']}")
print(f"Drawdown start: {stats['strategy'][strategy]['drawdown_start']}")
print(f"Drawdown end: {stats['strategy'][strategy]['drawdown_end']}")

# Load backtested trades as dataframe
trades = load_backtest_data(BACKTEST_RESULTS_DIR)

if len(trades) > 0:
    print("\nExit reasons per pair:")
    print(trades.groupby("pair")["exit_reason"].value_counts())

Results per pair:
[{'key': 'BTC/USDT:USDT', 'trades': 2, 'profit_mean': 0.0673275138235969, 'profit_mean_pct': 6.73, 'profit_sum': 0.1346550276471938, 'profit_sum_pct': 13.47, 'profit_total_abs': 137.19270638, 'profit_total': 0.13719270638, 'profit_total_pct': 13.72, 'duration_avg': '0:20:00', 'wins': 2, 'draws': 0, 'losses': 0, 'winrate': 1.0}, {'key': 'TOTAL', 'trades': 2, 'profit_mean': 0.0673275138235969, 'profit_mean_pct': 6.73, 'profit_sum': 0.1346550276471938, 'profit_sum_pct': 13.47, 'profit_total_abs': 137.19270638, 'profit_total': 0.13719270638, 'profit_total_pct': 13.72, 'duration_avg': '0:20:00', 'wins': 2, 'draws': 0, 'losses': 0, 'winrate': 1.0}]

Pairlist:
['BTC/USDT:USDT']

Market change: 0.025353203628543475
Drawdown start: 1970-01-01 00:00:00+00:00
Drawdown end: 1970-01-01 00:00:00+00:00

Exit reasons per pair:
pair           exit_reason
BTC/USDT:USDT  POC_Lower      1
               VWAP_Lower     1
Name: count, dtype: int64


## Plotting daily profit / equity line

In [7]:
import pandas as pd
import plotly.express as px

strategy_stats = stats["strategy"][strategy]

df = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
df["equity_daily"] = df["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily", title="CVDDivergence Strategy - Equity Curve")
fig.show()

## Plot results

In [8]:
from freqtrade.plot.plotting import generate_candlestick_graph

from_date = data["delta"].dropna().index.min()
to_date = data["delta"].dropna().index.max()

# Filter data for a specific date range
data_filtered = data[from_date:to_date].drop(columns=["exit_short", "exit_long"])

# Generate candlestick graph with indicators for trade entry and losses
graph = generate_candlestick_graph(
    pair=PAIR,
    data=data_filtered,
    trades=trades,
    indicators1=["vwap", "vwap_lower_2", "vwap_upper_2", "bull_poc_upper", "bull_poc_lower", "bear_poc_lower", "bear_poc_upper"],
    indicators2=["cvd_ma"]
)

# Render graph in a separate window
graph.show(renderer="browser")